In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF

from flux.form_factors import get_form_factor_matrix
from flux.shape import CgalTrimeshShapeModel, get_surface_normals, get_centroids
from flux.quadtree import get_quadrant_order
from flux.octree import get_octant_order

import pyvista as pv

# Regions of Interest

If we specify a region of interest (a circular region surrounding some position), we partition the space separate within and outside of the ROI. This could lead to better concentration of rank in the hierarchical view factor matrix.

In [69]:
import itertools as it

def get_quadrant_order_roi(X, roi_c, roi_r):
    
    roi_idx = (np.power(X - roi_c, 2).sum(axis=1) < np.power(roi_r, 2))
    
    X_roi = X[roi_idx]
    xmin_roi, ymin_roi = np.min(X_roi, axis=0)
    xmax_roi, ymax_roi = np.max(X_roi, axis=0)
    xc_roi, yc_roi = (xmin_roi + xmax_roi)/2, (ymin_roi + ymax_roi)/2
    
    X_non_roi = X[~roi_idx]
    xmin_non_roi, ymin_non_roi = np.min(X_non_roi, axis=0)
    xmax_non_roi, ymax_non_roi = np.max(X_non_roi, axis=0)
    xc_non_roi, yc_non_roi = (xmin_non_roi + xmax_non_roi)/2, (ymin_non_roi + ymax_non_roi)/2
        
    x, y = X.T
    Is = []
    for xop, yop in it.product([np.less_equal, np.greater], repeat=2):
        B = np.column_stack([xop(x, xc_roi), yop(y, yc_roi), roi_idx])
        I = np.where(np.all(B, axis=1))[0]
        Is.append(I)
        
        B = np.column_stack([xop(x, xc_roi), yop(y, yc_roi), ~roi_idx])
        I = np.where(np.all(B, axis=1))[0]
        Is.append(I)
    return Is

### Load the shape mesh and extract the two sets of quadrants.

Make sure the directory to the mesh is correct on your machine!

In [ ]:
mesh_dir = 'python-flux/examples/gerlache/'
mesh_name = 'gerlache'

In [79]:
max_inner_area = 0.8
max_outer_area = 3.0

max_inner_area_str = str(max_inner_area)
max_outer_area_str = str(max_outer_area)

In [80]:
V = np.load(f'{mesh_dir}/{mesh_name}_verts_{max_inner_area_str}_{max_outer_area_str}.npy')
F = np.load(f'{mesh_dir}/{mesh_name}_faces_{max_inner_area_str}_{max_outer_area_str}.npy')

# convert verts from km to m
V *= 1e3

N = get_surface_normals(V, F)
N[N[:, 2] > 0] *= -1

In [81]:
P = get_centroids(V, F)
Is = get_quadrant_order_roi(P[:,:2], V.mean(axis=0)[:2], 1e4)

In [82]:
faces_padded = np.concatenate([3*np.ones(F.shape[0],dtype=int).reshape(-1,1), F], axis=1)
grid = pv.PolyData(V, faces=faces_padded.flatten(), n_faces=F.shape[0])

shape_model = CgalTrimeshShapeModel(V, F, N)

In [83]:
num_faces = F.shape[0]
print('created mesh with %d triangles' % num_faces)

created mesh with 2078 triangles


In [84]:
grid['quadrant'] = np.empty(P.shape[0])
grid['quadrant'][...] = np.nan
for i, I in enumerate(Is):
    grid['quadrant'][I] = i

In [85]:
grid.plot()

Widget(value="<iframe src='http://localhost:61510/index.html?ui=P_0x1ee5ab3ddd0_3&reconnect=auto' style='width…

View factor between two blocks in the crater is very dense

In [96]:
I = Is[0]
J = Is[2]
FF = get_form_factor_matrix(shape_model, I, J)
FF_arr = FF.A
print(FF.shape, FF.nnz)
print(FF.nnz/np.prod(FF.shape))

(172, 177) 29608
0.972539745105768


View factor between a block inside the crater and a block outside the crater is very sparse

In [104]:
I = Is[0]
J = Is[5]
FF = get_form_factor_matrix(shape_model, I, J)
FF_arr = FF.A
print(FF.shape, FF.nnz)
print(FF.nnz/np.prod(FF.shape))

(172, 348) 318
0.005312750601443464


View factor between two blocks outside the crater is fairly sparse

In [91]:
I = Is[1]
J = Is[3]
FF = get_form_factor_matrix(shape_model, I, J)
FF_arr = FF.A
print(FF.shape, FF.nnz)
print(FF.nnz/np.prod(FF.shape))

(346, 355) 4698
0.038247985019946265


In [105]:
I = Is[1]
J = Is[1]
FF = get_form_factor_matrix(shape_model, I, J)
FF_arr = FF.A
print(FF.shape, FF.nnz)
print(FF.nnz/np.prod(FF.shape))

(346, 346) 14576
0.12175481974005145
